__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/zemelnye_uchastki/prodam/izhs'
    
raw_data_path = 'data/raw/land/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-15 10:30:11 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-15 10:30:11 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-15 10:30:13 | AdsListParser: downloader init
[INFO    ] 2022-09-15 10:30:13 | AvitoParser: init
[INFO    ] 2022-09-15 10:30:18 | AvitoParser: 23 pages for read
[INFO    ] 2022-09-15 10:30:18 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-15 10:30:21 | AdsListParser: read page 1
[INFO    ] 2022-09-15 10:30:25 | AdsListParser: read page 2
[INFO    ] 2022-09-15 10:30:29 | AdsListParser: read page 3
[INFO    ] 2022-09-15 10:30:32 | AdsListParser: read page 4
[INFO    ] 2022-09-15 10:30:35 | AdsListParser: read page 5
[INFO    ] 2022-09-15 10:30:38 | AdsListParser: read page 6
[INFO    ] 2022-09-15 10:30:41 | AdsListParser: read page 7
[INFO    ] 2022-09-15 10:30:45 | AdsListParser: read page 8
[INFO    ] 2022-09-15 10:30:48 | AdsListParser: read page 9
[INFO    ] 2022-09-15 10:30:51 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

1127
1121


In [ ]:
df['place']=place

In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [9]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
528,2317736674,Объявление «Участок 7 сот. (ИЖС)»,2700000,,"Нахимовский муниципальный округ, посёлок Любимовка","Продам свой участок, свет подведен на участок, газ и вода можно подвести. \n\nТорг с реальным покупателем.\n\nАгентов просьба не беспокоить.",11,2022-09-15 10:31:26.038609
78,2399408010,Объявление «Участок 11 сот. (ИЖС)» 3 фотографии,4000000,,"с. Тыловое, Ялтинская ул., 2","СРОЧНО.СОБСТВЕННИК.Продам шикарный ровный участок под ИЖС ,с видом на горы в Байдарской долине.Рядом два живописных озера,отличные подъезд,5 минут пешком до Ялтинской трассы.Коммуникации вдоль участка.Документы,госакт в наличии.Без посредника,агенства прош",2,2022-09-15 10:31:26.038609
